In [ ]:
"""
Goal:
For a given |Ψ> and Gamma for 2 qubits
reproduce figure 3 by compute H' and S and solve HC=SCE
"""

In [ ]:
import numpy as np
import math
from collections import Counter

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, assemble, QuantumRegister, ClassicalRegister, BasicAer, execute
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.aer import QasmSimulator
from qiskit.opflow import I,Z,X,Y,StateFn

from scipy.linalg import eigh, eig
from collections.abc import Iterable   # import directly from collections for Python < 3.3

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

In [ ]:
def real(M):
    if isinstance(M, Iterable):
        return [real(Mi) for Mi in M]
    else:
        return M.real
def format_matrix(matrix):
    s = [[str(e) for e in row] for row in matrix]
    lens = [max(map(len, col)) for col in zip(*s)]
    fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
    table = [fmt.format(*row) for row in s]
    return '\n'.join(table)

In [ ]:
# [5,1,3]
generators = [X^Z^Z^X^I, I^X^Z^Z^X, X^I^X^Z^Z, Z^X^I^X^Z] # M's

projectors = [((I^I^I^I^I) + g)/2 for g in generators]

projector = I^I^I^I^I
for p in projectors:
    projector = projector @ p

statefn_a = StateFn('00001')
statefn = projector @ statefn_a
# TODO: force it to be logical 0, and then add an error

# now we have a state that's in the +1 eigenstate of all the generators so it's in the codespace
# normalize the state:
norm = (statefn.adjoint() @ statefn).eval()
statefn = statefn / (norm ** 0.5)
print("norm of the normalized state = ", (statefn.adjoint() @ statefn).eval())
# check we are getting 1 for all
print("expectation value of all generators for the state:", [(statefn.adjoint() @ g @ statefn).eval() for g in generators])

Ms = [p * (1 / norm) for p in projector]
print("There are ", len(Ms), "Ms:")
print(Ms)

Sij = []
for i in range(len(Ms)):
    Sij.append([])
    for j in range(len(Ms)):
        Sij[-1].append((statefn.adjoint() @ Ms[i] @ Ms[j] @ statefn).eval())

print("Sij=")
print(format_matrix(real(Sij)))
# If there is no error, all entries in Sij should be 1's

H = -sum(Ms)

Hij = []
for i in range(len(Ms)):
    Hij.append([])
    for j in range(len(Ms)):
        Hij[-1].append((statefn.adjoint() @ Ms[i] @ H @ Ms[j] @ statefn).eval())
print("Hij=")
print(format_matrix(real(Hij)))

# now solve for HC=SCE then compute gamma

# do we project to encode a|l0> + b|l1> then apply an error then project again?

# classically compute <gamma>
# classically compute <gamma> with no errors from the beginning
# compute fidelity and plot the chart for different amount_of_noise*

norm of the normalized state =  (1+0j)
expectation value of all generators for the state: [(1+0j), (1+0j), (1+0j), (1+0j)]
There are  16 Ms:
[PauliSumOp(SparsePauliOp(['IIIII'],
              coeffs=[1.+0.j]), coeff=1.0), PauliSumOp(SparsePauliOp(['ZXIXZ'],
              coeffs=[1.+0.j]), coeff=1.0), PauliSumOp(SparsePauliOp(['XIXZZ'],
              coeffs=[1.+0.j]), coeff=1.0), PauliSumOp(SparsePauliOp(['YXXYI'],
              coeffs=[1.+0.j]), coeff=1.0), PauliSumOp(SparsePauliOp(['IXZZX'],
              coeffs=[1.+0.j]), coeff=1.0), PauliSumOp(SparsePauliOp(['ZIZYY'],
              coeffs=[1.+0.j]), coeff=1.0), PauliSumOp(SparsePauliOp(['XXYIY'],
              coeffs=[1.+0.j]), coeff=1.0), PauliSumOp(SparsePauliOp(['YIYXX'],
              coeffs=[1.+0.j]), coeff=1.0), PauliSumOp(SparsePauliOp(['XZZXI'],
              coeffs=[1.+0.j]), coeff=1.0), PauliSumOp(SparsePauliOp(['YYZIZ'],
              coeffs=[1.+0.j]), coeff=1.0), PauliSumOp(SparsePauliOp(['IZYYZ'],
              coeffs=[

In [ ]:
# from https://stackoverflow.com/questions/24752393/solve-generalized-eigenvalue-problem-in-numpy
# docs: https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eig.html#scipy.linalg.eig
# eigh(Hij, Sij, eigvals_only=False)#, subset_by_index=[0,1])
E,C = eig(Hij, Sij)
print("E=",real(E))
print("C=\n", format_matrix(real(C)))

E= [-16.000000000000007, -16.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
C=
 -1.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0 	1.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0 	0.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0 	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0 	0.0	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0 	0.0	0.0	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0 	0.0	0.0	0.0	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0 	0.0	0.0	0.0	0.0	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0 	0.0	0.0	0.0	0.0	0.0	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0 	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0 	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0
0.0 	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	1.0	0.0	0.0	0.0	0.0
0.0 	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	1.0	0.0	0.0	0.0
0.0 	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	